In [3]:
# import necessary libraries
import numpy as np
import pandas as pd
import bs4 as bs
import requests
import json

<h1> Step 1: Obtain the list of S&P 500 Stocks from a wikipedia page

In [4]:
html = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(html.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

In [5]:
# helper function
def clean_list(strings):
    new_string = [string[:-1] if string.endswith('\n') else string for string in strings]
    cleaned_string = [string.replace('\n','') if ('\n' in string) else string for string in new_string]
    return cleaned_string


In [6]:
tickers = []
stock_name = []
sector = []
sub_industry = []
hq_location = []
date_added = []
cik = []
founded = []

data = [tickers,stock_name, sector, sub_industry, hq_location, date_added, cik, founded]
for row in table.findAll('tr')[1:]:
    for idx, data_category in enumerate(data):
        data_value = row.findAll('td')[idx].text
#         data_value = data_value[:-1]
        data_category.append(data_value)

tickers = clean_list(tickers)
cik = clean_list(cik)
founded = clean_list(founded)

In [7]:
stocks_data_df = pd.DataFrame({
                                'tickers': tickers,
                                'stock_name': stock_name,
                                'sector': sector,
                                'sub_industry': sub_industry,
                                'hq_location': hq_location,
                                'date_added': date_added,
                                'cik': cik,
                                'founded': founded
                                }
                                  )

In [8]:
stocks_data_df.head()

,tickers,stock_name,sector,sub_industry,hq_location,date_added,cik,founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,0000066740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,0000001800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989


<h1> Step 2: Build the data by obtaining market information for each of the S&P Stocks</h1>
<b>Data Sources</b>: <a href>www.stockanalysis.com</a>

<h3> Helper Function: Merge Function</h3>

In [132]:
'''
A function to merge two dictionaries with the same format into a new dictionary with combined data.

Arguments:
- dict1: A dictionary containing data to merge.
- dict2: A dictionary containing data to merge.

Returns:
A new dictionary with the merged data.
The merged dictionary will have the same column headings as `dict1` and `dict2`, with duplicates removed. 
The rows of the merged data will be formed by concatenating the rows with the same heading from `dict1` and `dict2`.
If a column heading exists in one dictionary but not the other, the missing values will be replaced with '-'.

'''

def merge_dicts(dict1, dict2):
    merged_dict = {'headings': [], 'data': []}

    # merge headings
    for heading in dict1['headings']:
        if heading not in merged_dict['headings']:
            merged_dict['headings'].append(heading)
    for heading in dict2['headings']:
        if heading not in merged_dict['headings']:
            merged_dict['headings'].append(heading)

    # merge data
    for i in range(len(merged_dict['headings'])):
        heading = merged_dict['headings'][i]
        if heading in dict1['headings']:
            idx1 = dict1['headings'].index(heading)
            data1 = dict1['data'][idx1]
        else:
            data1 = ['-'] * len(dict1['data'][0])
        if heading in dict2['headings']:
            idx2 = dict2['headings'].index(heading)
            data2 = dict2['data'][idx2]
        else:
            data2 = ['-'] * len(dict2['data'][0])
        merged_data = data1 + data2
        merged_dict['data'].append(merged_data)

    return merged_dict


<h3> Obtain the annual financial data for the stocks for a maximum of 10 years </h3>

In [175]:
import time
# now fetch the data and combine for all tickers
# years = ['2022','2021','2020','2019','2018','2017','2016','2015','2014','2013'] #10 years data
whole_market_data = {'headings':[], 'data': []}
# test_stocks = ['ICE','IVZ','MKC','ZBH','ZION','ZTS']
# test_stocks = ['AMCR','BKR','CZR','CARR','CDAY','CFG','CEG','FRC','FSLR','FE','FIS','INTU']
# test_stocks = ['ICE','AMCR','LVS','LDOS','LEN','LNC','LYV','LOW','TAP','MDLZ','OGN','VFC','YUM']
missing_stocks = []
copied_stocks = []
for i_tick,ticker in enumerate(stocks_data_df['tickers']):
# for i_tick,ticker in enumerate(test_stocks):
    headings = []
    headings_with_year = [] 
    segmented_data = []    
    
    if i_tick % 20 == 0:
        time.sleep(30) #sleep for 20 seconds after scraping every 50 stocks
    else:
        time.sleep(5)
    
    financials_link = f"https://stockanalysis.com/stocks/{ticker}/financials/"
    html = requests.get(financials_link)
    soup = bs.BeautifulSoup(html.text, 'lxml')
    table = soup.find('table', {'class': 'w-full whitespace-nowrap'})
    
    
    # Find the thead tag within the table
    try:
        table_head = table.find('thead')
    except:
        
        print(ticker)

    # Extract all th tags (table headings) within the thead tag
    table_headings = []
    for th in table_head.find_all('th'):
        table_headings.append(th.text.strip())
    years = [x for x in table_headings if all(c.isdigit() for c in str(x))]
    
    column_index_to_skip = [i for i, x in enumerate(table_headings) if x not in years]
    
    whole_data = []
    try:
        for row in table.findAll('tr')[1:]:
            index = 0
            new_data = []
            while(index < len(table_headings)):
                data_value = row.findAll('td')[index].text
                new_data.append(data_value)
                index += 1
            whole_data.append(clean_list(new_data))
    except Exception as e:
        print(e)
        print(ticker)
        missing_stocks.append(ticker)
    headings = [data_head[0] for data_head in whole_data] 

    for idy,heading in enumerate(headings): 
        for idx,year in enumerate(years): 
            headings_with_year.append(heading+' ('+ year + ')') 
            
    whole_data_index = 0
    for idx, row in enumerate(whole_data):
        for idy, column in enumerate(row):
            if idy in column_index_to_skip: #skipping the first and last columns in most cases
                # also make allowances for special cases
                continue
            else:
                try:
                    segmented_data.append([column])
                    whole_data_index +=1
                except:
                    print(f'Exception caught at {column} ')
    new_market_data = {'headings':headings_with_year, 'data': segmented_data}
    copied_stocks.append(ticker)
    if i_tick == 0:
        whole_market_data = new_market_data
#         print((new_market_data['headings']))
#         print((new_market_data['data']))
    else:
#         print((new_market_data['headings']))
#         print((new_market_data['data']))
        try:
            whole_market_data = merge_dicts(whole_market_data,new_market_data)
        except:
            print("Error in this iteration: "+ticker)
# whole_market_data

<h4> Do data integrity check by checking the size of the data</h4>

In [176]:
# data integrity checks
# 1. check the length of the stocks
# 2. check the amount of data in each data list
print(len(whole_market_data['headings']))
print(len(whole_market_data['data']))

469
469


In [177]:
len(whole_market_data['data'][6])

503

In [178]:
len(whole_market_data['data'][16])

503

In [179]:
data_count_list = [len(whole_market_data['data'][i]) for i in range(469)]

In [181]:
print(min(data_count_list))
print(max(data_count_list))

503
503


In [183]:
complete_stock_data_df = stocks_data_df.copy()

In [184]:
complete_stock_data_df.head()

,tickers,stock_name,sector,sub_industry,hq_location,date_added,cik,founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,0000066740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,0000001800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989


<h4> Append the new data to the initial dataframe for the data obtained from the wikipedia page</h4>

In [189]:
# len(complete_stock_data_df)
for idx, headings in enumerate(whole_market_data['headings']):
    complete_stock_data_df[headings] = (whole_market_data['data'])[idx]

In [193]:
complete_stock_data_df[complete_stock_data_df['tickers'] == 'TSLA']

,tickers,stock_name,sector,sub_industry,hq_location,date_added,cik,founded,Revenue (2022),Revenue (2021),...,Gross Margin (2011),Operating Margin (2011),Profit Margin (2011),Free Cash Flow Margin (2011),Effective Tax Rate (2011),EBITDA (2011),EBITDA Margin (2011),Depreciation & Amortization (2011),EBIT (2011),EBIT Margin (2011)
445,TSLA,"Tesla, Inc.",Consumer Discretionary,Automobile Manufacturers,"Austin, Texas",2020-12-21,0001318605,2003,"81,462","53,823",...,-,-,-,-,-,-,-,-,-,-


In [194]:
# save file for later
# save DataFrame as a CSV file
complete_stock_data_df.to_csv('complete_stock_data.csv', index=False)

<h4> Create a function for the scrapping function </h4>

In [216]:
# also scrap for Balance sheet, cash flow and ratios tables
import time

'''
This function scrapes financial data from the StockAnalysis website for a list of tickers, 
and returns a dictionary with the data organized by year. 
The function takes one required argument, link_extension, which specifies the type of 
financial data to scrape (balance sheet, cash flow, ratios). 

The function also takes an optional argument, tickers, which is a list of stock tickers to scrape data for.
If no list is provided, the function will use the tickers column from a pandas DataFrame called stocks_data_df. 
The function starts by initializing an empty dictionary called whole_market_data. 

For each ticker in the list of tickers, the function sends a GET request to the financials page for that ticker, 
extracts the table data, and organizes it into a dictionary called new_market_data. 
If this is the first ticker, new_market_data is assigned to whole_market_data. 
Otherwise, the merge_dicts function is used to merge new_market_data into whole_market_data. 

Finally, the function returns whole_market_data.

Note that the function includes some error handling, such as sleeping for 20 seconds after scraping every 50 stocks, 
and printing any tickers that result in errors during scraping. 
The function also includes a helper function called clean_list that removes unnecessary whitespace from strings in a list.

'''
def get_table_data(link_extension='financials/',tickers=stocks_data_df['tickers']):
    whole_market_data = {'headings':[], 'data': []}
    missing_stocks = []
    copied_stocks = []
    for i_tick,ticker in enumerate(tickers):
    # for i_tick,ticker in enumerate(test_stocks):
        headings = []
        headings_with_year = [] 
        segmented_data = []    

        if i_tick % 20 == 0:
            time.sleep(30) #sleep for 20 seconds after scraping every 50 stocks
        else:
            time.sleep(5)

        financials_link = f"https://stockanalysis.com/stocks/{ticker}/{link_extension}"
        html = requests.get(financials_link)
        soup = bs.BeautifulSoup(html.text, 'lxml')
        table = soup.find('table', {'class': 'w-full whitespace-nowrap'})


        # Find the thead tag within the table
        try:
            table_head = table.find('thead')
        except:

            print(ticker)

        # Extract all th tags (table headings) within the thead tag
        table_headings = []
        for th in table_head.find_all('th'):
            table_headings.append(th.text.strip())
        years = [x for x in table_headings if all(c.isdigit() for c in str(x))]

        column_index_to_skip = [i for i, x in enumerate(table_headings) if x not in years]

        whole_data = []
        try:
            for row in table.findAll('tr')[1:]:
                index = 0
                new_data = []
                while(index < len(table_headings)):
                    try:
                        data_value = row.findAll('td')[index].text
                    except IndexError: # ['ALK','AAL','AWK','APTV','BA','CCL','DAL','DIS','DXC','EXPE','FICO','GNRC','LVS','MTCH','PCG','RCL'
                    #,'TMUS','USB','WDC','WYNN'] in the ratios table ---making an exception to properly handle the case
                        data_value = '-'
                        
                    new_data.append(data_value)
                    index += 1
                whole_data.append(clean_list(new_data))
            
        except Exception as e:
            print(e)
            print(ticker)
            missing_stocks.append(ticker)
        headings = [data_head[0] for data_head in whole_data] 

        for idy,heading in enumerate(headings): 
            for idx,year in enumerate(years): 
                headings_with_year.append(heading+' ('+ year + ')') 

        whole_data_index = 0
        for idx, row in enumerate(whole_data):
            for idy, column in enumerate(row):
                if idy in column_index_to_skip: #skipping the first and last columns in most cases
                    # also make allowances for special cases
                    continue
                else:
                    try:
                        segmented_data.append([column])
                        whole_data_index +=1
                    except:
                        print(f'Exception caught at {column} ')
        new_market_data = {'headings':headings_with_year, 'data': segmented_data}
        copied_stocks.append(ticker)
        if i_tick == 0:
            whole_market_data = new_market_data
    #         print((new_market_data['headings']))
    #         print((new_market_data['data']))
        else:
    #         print((new_market_data['headings']))
    #         print((new_market_data['data']))
            try:
                whole_market_data = merge_dicts(whole_market_data,new_market_data)
            except:
                print("Error in this iteration: "+ticker)
    # whole_market_data  
    return whole_market_data

<h4> Using the <b> get_table_data </b> function, obtain the data for the balance sheet, cash flow and ratios </h4>

In [196]:
balance_sheet_data = get_table_data(link_extension='financials/balance-sheet/',tickers=stocks_data_df['tickers'])

In [197]:
cash_flow_data = get_table_data(link_extension='financials/cash-flow-statement/',tickers=stocks_data_df['tickers'])

In [240]:
ratios_data = get_table_data(link_extension='financials/ratios/',tickers=stocks_data_df['tickers'])

<h4> Data integrity checks </h4>

In [241]:
# Data integrity checks
print(len(ratios_data['headings']))
print(len(ratios_data['data']))
data_count_list = [len(ratios_data['data'][i]) for i in range(len(ratios_data['data']))]
print(min(data_count_list))
print(max(data_count_list))

286
286
503
503


In [221]:
# now we fix the issue of ratios
interested_stocks = ['TSLA','ALK','AAL','AWK','APTV','BA','CCL','DAL','DIS','DXC','EXPE','FICO','GNRC','LVS','MTCH','PCG','RCL'
                    ,'TMUS','USB','WDC','WYNN']
reduced_ratios_data = get_table_data(link_extension='financials/ratios/',tickers=interested_stocks)
print(len(reduced_ratios_data['headings']))
print(len(reduced_ratios_data['data']))
data_count_list = [len(reduced_ratios_data['data'][i]) for i in range(len(reduced_ratios_data['data']))]
print(min(data_count_list))
print(max(data_count_list))

286
286
21
21


In [199]:
# Do data integrity checks

# data integrity checks
# 1. check the length of the stocks
# 2. check the amount of data in each data list
print(len(balance_sheet_data['headings']))
print(len(balance_sheet_data['data']))

405
405


In [200]:
data_count_list = [len(balance_sheet_data['data'][i]) for i in range(len(balance_sheet_data['data']))]
print(min(data_count_list))
print(max(data_count_list))

503
503


In [201]:
print(len(cash_flow_data['headings']))
print(len(cash_flow_data['data']))
data_count_list = [len(cash_flow_data['data'][i]) for i in range(len(cash_flow_data['data']))]
print(min(data_count_list))
print(max(data_count_list))

272
272
503
503


In [203]:
complete_stock_data_extra_df = complete_stock_data_df.copy()

<h4> Add the new data to the existing data set</h4>

In [209]:
# Add balance sheet data to existing dataframe
for idx, headings in enumerate(balance_sheet_data['headings']):
    complete_stock_data_extra_df[headings] = (balance_sheet_data['data'])[idx]
print((complete_stock_data_extra_df.shape))

(503, 882)


In [212]:
# Add cash flow data to existing dataframe
for idx, headings in enumerate(cash_flow_data['headings']):
    complete_stock_data_extra_df[headings] = (cash_flow_data['data'])[idx]
print((complete_stock_data_extra_df.shape))

(503, 1106)


In [214]:
# save file for later
# save DataFrame as a CSV file
complete_stock_data_extra_df.to_csv('complete_stock_extra_data.csv', index=False)

In [244]:
complete_stock_data_extra_with_ratios_df = complete_stock_data_extra_df.copy()

# Add ratios data to existing dataframe
for idx, headings in enumerate(ratios_data['headings']):
    complete_stock_data_extra_with_ratios_df[headings] = (ratios_data['data'])[idx]

<ipython-input-244-8d1e40dc7afe>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_stock_data_extra_with_ratios_df[headings] = (ratios_data['data'])[idx]
<ipython-input-244-8d1e40dc7afe>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_stock_data_extra_with_ratios_df[headings] = (ratios_data['data'])[idx]
<ipython-input-244-8d1e40dc7afe>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all column

<ipython-input-244-8d1e40dc7afe>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_stock_data_extra_with_ratios_df[headings] = (ratios_data['data'])[idx]
<ipython-input-244-8d1e40dc7afe>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_stock_data_extra_with_ratios_df[headings] = (ratios_data['data'])[idx]
<ipython-input-244-8d1e40dc7afe>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all column

(503, 1392)


<ipython-input-244-8d1e40dc7afe>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_stock_data_extra_with_ratios_df[headings] = (ratios_data['data'])[idx]
<ipython-input-244-8d1e40dc7afe>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  complete_stock_data_extra_with_ratios_df[headings] = (ratios_data['data'])[idx]
<ipython-input-244-8d1e40dc7afe>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all column

In [245]:
print((complete_stock_data_extra_with_ratios_df.shape))

(503, 1392)


In [246]:
# save file for later
# save DataFrame as a CSV file
complete_stock_data_extra_with_ratios_df.to_csv('complete_stock_extra_data_with_ratios.csv', index=False)

<h2> Features Engineering</h2>


Some essential metrics are not available but they can be computed from the existing data.

In [280]:
# creating new features
'''
1.Inventory turnover
- it is much need. However, only the ratio for the current year is provided
- Inventory turnover = Cost of Goods Sold / Average inventory
- Cost of Goods Sold = Cost of Sales = Cost of Revenue (it is the cost associated with 
                                                            producing and selling a company's products)

'''
# for each stock, fetch the years for which we have their data
# loop through and compute the metric
# collate and add the new information to the dataframe


# another angle, use the whole column for the big table to filter years
# for each stock, check if the data for that year is available,
# if it is, compute the value
# if not, represent with a dash

# let's take the first stock and extract the years out of it
import re

# new_df = complete_stock_data_extra_with_ratios_df[complete_stock_data_extra_with_ratios_df["tickers"] == complete_stock_data_extra_with_ratios_df["tickers"][0]]
unique_years = sorted(list(set(re.findall(r'\d+', ' '.join(complete_stock_data_extra_with_ratios_df.columns.tolist())))),reverse=True)
print(unique_years)

# Cost of Revenue
# Inventory

for single_year in unique_years:
    column_name = "Inventory turnover ("+single_year+")"
    new_inv_turnover_list = []
    try:
        cost_of_revenue = (complete_stock_data_extra_with_ratios_df['Cost of Revenue ('+single_year+')']).tolist()
        average_inventory = (complete_stock_data_extra_with_ratios_df['Inventory ('+single_year+')']).tolist()
    except:
        print('Data for {} not available'.format(single_year))
    for iTick,ticker in enumerate(complete_stock_data_extra_with_ratios_df['tickers']):
        try:
            tick_new_inv_turnover = float(cost_of_revenue[iTick].replace(',', '')) / float(average_inventory[iTick].replace(',', ''))
            tick_new_inv_turnover = round(tick_new_inv_turnover, 4)
        except:
            tick_new_inv_turnover = '-'     
        new_inv_turnover_list.append(tick_new_inv_turnover)
#     print(new_inv_turnover_list)
#     print(column_name)
    complete_stock_data_extra_with_ratios_df[column_name] = new_inv_turnover_list #added new features to dataframe

['2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011']
Data for 2011 not available


In [282]:
'''
Add more features to the table
Compute cash ratio and Quick ratio

Cash ratio = (cash and cash equivalents) / current liabilities

Quick Ratio = (Current Assets - (Inventories + Other Current Assets)) / Current Liabilities

D/C ratio (Debt to capital ratio) = Total Debt / (Total Debt + Total Equity)
Total Equity = Shareholders' Equity
Receivables turnover = Revenue / Receivables
Return on assets (ROA) = Net income / Total assets
Return on equity (ROE) = Net income / Total equity

'''

"\nAdd more features to the table\nCompute cash ratio and Quick ratio\n\nCash ratio = (cash and cash equivalents) / current liabilities\n\nQuick Ratio = (Current Assets - Inventories) / Current Liabilities\n\nD/C ratio (Debt to capital ratio) = Total Debt / (Total Debt + Total Equity)\nTotal Equity = Shareholders' Equity\nReceivables turnover = Revenue / Receivables\nReturn on assets (ROA) = Net income / Total assets\nReturn on equity (ROE) = Net income / Total equity\n\n"

In [308]:
unique_years = sorted(list(set(re.findall(r'\d+', ' '.join(complete_stock_data_extra_with_ratios_df.columns.tolist())))),reverse=True)
print(unique_years)

# Cost of Revenue
# Inventory

for single_year in unique_years:    
    column_name_cash = "Cash ratio ("+single_year+")"
    column_name_quick = "Quick ratio ("+single_year+")"
    column_name_roe = "ROE ("+single_year+")"
    column_name_roa = "ROA ("+single_year+")"
    column_name_d_c = "D/C ratio ("+single_year+")"
    column_name_rec_turnover = "Receivables turnover ("+single_year+")"

    new_cash_ratio_list = []
    new_quick_ratio_list = []
    new_roe_list = []
    new_roa_list = []
    new_d_c_ratio_list = []
    new_rec_turnover_list = []
    
    try:
        cash_and_cash_equivalent = (complete_stock_data_extra_with_ratios_df['Cash & Cash Equivalents ('+single_year+')']).tolist()
        current_liabilities = (complete_stock_data_extra_with_ratios_df['Total Current Liabilities ('+single_year+')']).tolist()
        inventory = (complete_stock_data_extra_with_ratios_df['Inventory ('+single_year+')']).tolist()
        current_asset = (complete_stock_data_extra_with_ratios_df['Total Current Assets ('+single_year+')']).tolist()
        total_debt = (complete_stock_data_extra_with_ratios_df['Total Debt ('+single_year+')']).tolist()
        total_equity = (complete_stock_data_extra_with_ratios_df["Shareholders' Equity ("+single_year+')']).tolist()
        revenue = (complete_stock_data_extra_with_ratios_df['Revenue ('+single_year+')']).tolist()
        receivables = (complete_stock_data_extra_with_ratios_df['Receivables ('+single_year+')']).tolist()
        total_assets = (complete_stock_data_extra_with_ratios_df['Total Assets ('+single_year+')']).tolist()
        net_income = (complete_stock_data_extra_with_ratios_df['Net Income ('+single_year+')']).tolist()
        other_current_assets = (complete_stock_data_extra_with_ratios_df['Other Current Assets ('+single_year+')']).tolist()
    except:
        print('Data for {} not available'.format(single_year))
    for iTick,ticker in enumerate(complete_stock_data_extra_with_ratios_df['tickers']):
        try:
            # Compute cash ratio
            tick_cash_ratio = float(cash_and_cash_equivalent[iTick].replace(',', '')) / float(current_liabilities[iTick].replace(',', ''))
            tick_cash_ratio = round(tick_cash_ratio, 4)
        except:
            tick_cash_ratio = '-'
            
        try:
            # Compute quick ratio
            tick_quick_ratio = (float(current_asset[iTick].replace(',', '')) - float(inventory[iTick].replace(',', '')) - float(other_current_assets[iTick].replace(',', '')) ) / float(current_liabilities[iTick].replace(',', ''))
            tick_quick_ratio = round(tick_quick_ratio, 4)
        except:
            tick_quick_ratio = '-'
            
        try:
            # Compute ROE
            tick_roe = (float(net_income[iTick].replace(',', '')) / float(total_equity[iTick].replace(',', ''))) * 100
            tick_roe = round(tick_roe, 4)
        except:
            tick_roe = '-'
        
        try:
            # Compute ROA
            tick_roa = (float(net_income[iTick].replace(',', '')) / float(total_assets[iTick].replace(',', ''))) * 100
            tick_roa = round(tick_roa, 4)
        except:
            tick_roa = '-'
            
        try:
            # Compute D/C ratio
            tick_d_c_ratio = float(total_debt[iTick].replace(',', '')) / (float(total_debt[iTick].replace(',', '')) + float(total_equity[iTick].replace(',', '')))
            tick_d_c_ratio = round(tick_d_c_ratio,4)
        except:
            tick_d_c_ratio = '-'
            
        try:
            # Receivables Turnover calculation
            tick_receivables_turnover = float(revenue[iTick].replace(',', '')) / float(receivables[iTick].replace(',', ''))
            tick_receivables_turnover = round(tick_receivables_turnover, 4)

        except:
            tick_receivables_turnover = '-'
            
            
            
        new_cash_ratio_list.append(tick_cash_ratio)
        new_rec_turnover_list.append(tick_receivables_turnover)
        new_d_c_ratio_list.append(tick_d_c_ratio)
        new_roa_list.append(tick_roa)
        new_roe_list.append(tick_roe)
        new_quick_ratio_list.append(tick_quick_ratio)
        
#     print(new_cash_ratio_list)
#     print(column_name)
    complete_stock_data_extra_with_ratios_df[column_name_cash] = new_cash_ratio_list #added new features to dataframe
    complete_stock_data_extra_with_ratios_df[column_name_quick] = new_quick_ratio_list #added new features to dataframe
    complete_stock_data_extra_with_ratios_df[column_name_roe] = new_roe_list #added new features to dataframe
    complete_stock_data_extra_with_ratios_df[column_name_roa] = new_roa_list #added new features to dataframe
    complete_stock_data_extra_with_ratios_df[column_name_d_c] = new_d_c_ratio_list #added new features to dataframe
    complete_stock_data_extra_with_ratios_df[column_name_rec_turnover] = new_rec_turnover_list #added new features to dataframe

['2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011']
Data for 2011 not available


In [309]:
complete_stock_data_extra_with_ratios_df.to_csv('whole_stock_data_with_added_features.csv', index=False)

In [168]:
import requests

url = 'https://www.stockanalysis.com/robots.txt'
response = requests.get(url)

print(response.text)


User-agent: *
Disallow:

User-agent: dotbot
Disallow: /

User-agent: BLEXBot
Disallow: /

User-agent: mj12bot
Disallow: /

Sitemap: https://stockanalysis.com/sitemap.xml
